<a href="https://colab.research.google.com/github/Hanbin-git/dacon_new_drug/blob/main/%EC%83%88%EB%A1%9C%EC%9A%B4%EC%A0%84%EB%9E%B5%20%EC%A0%81%EC%9A%A9%ED%95%9C%20%EA%B2%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import zipfile

zip_path = '/content/drive/MyDrive/data.zip'
extract_path = '/content/project_data'  # 원하는 경로

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [3]:
import pandas as pd

path = '/content/project_data/'  # 압축 해제 경로
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

print(train.shape, test.shape, submission.shape)


(1681, 3) (100, 2) (100, 2)


In [4]:
# RDKit 설치 (Colab에서 가능)
!pip uninstall -y rdkit-pypi
!pip install rdkit-pypi==2022.9.5 optuna xgboost



Found existing installation: rdkit-pypi 2022.9.5
Uninstalling rdkit-pypi-2022.9.5:
  Successfully uninstalled rdkit-pypi-2022.9.5
  Using cached rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.9 kB)
Using cached rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (29.4 MB)


In [5]:
# # ✅ RDKit + XGBoost + Optuna 전체 파이프라인
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error
# from rdkit import Chem
# from rdkit.Chem import Descriptors, Lipinski, MACCSkeys, AllChem
# import xgboost as xgb
# import optuna
# import warnings
# warnings.filterwarnings("ignore")

# # ✅ 경로 설정
# path = '/content/project_data/'
# train = pd.read_csv(path + 'train.csv')
# test = pd.read_csv(path + 'test.csv')
# submission = pd.read_csv(path + 'sample_submission.csv')

# # ✅ 분자 특성 추출 함수
# def get_molecule_descriptors(smiles):
#     try:
#         mol = Chem.MolFromSmiles(smiles)
#         if mol is None: return [0] * 2232
#         basic = [
#             Descriptors.MolWt(mol), Descriptors.MolLogP(mol),
#             Descriptors.NumHAcceptors(mol), Descriptors.NumHDonors(mol),
#             Descriptors.TPSA(mol), Descriptors.NumRotatableBonds(mol),
#             Descriptors.NumAromaticRings(mol), Descriptors.NumHeteroatoms(mol),
#             Descriptors.FractionCSP3(mol), Descriptors.NumAliphaticRings(mol),
#             Lipinski.NumAromaticHeterocycles(mol), Lipinski.NumSaturatedHeterocycles(mol),
#             Lipinski.NumAliphaticHeterocycles(mol), Descriptors.HeavyAtomCount(mol),
#             Descriptors.RingCount(mol), Descriptors.NOCount(mol),
#             Descriptors.NHOHCount(mol), Descriptors.NumRadicalElectrons(mol)
#         ]
#         morgan = [int(b) for b in AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048).ToBitString()]
#         maccs = [int(b) for b in MACCSkeys.GenMACCSKeys(mol).ToBitString()]
#         return basic + morgan + maccs
#     except:
#         return [0] * 2232

# # ✅ 피처 생성
# train['features'] = train['Canonical_Smiles'].apply(get_molecule_descriptors)
# test['features'] = test['Canonical_Smiles'].apply(get_molecule_descriptors)
# X_train = np.array(train['features'].tolist())
# y_train = train['Inhibition'].values
# X_test = np.array(test['features'].tolist())

# # ✅ 정규화
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # ✅ 평가 함수
# def normalized_rmse(y_true, y_pred):
#     return np.sqrt(mean_squared_error(y_true, y_pred)) / (np.max(y_true) - np.min(y_true))
# def pearson_correlation(y_true, y_pred):
#     return np.clip(np.corrcoef(y_true, y_pred)[0, 1], 0, 1)
# def competition_score(y_true, y_pred):
#     return 0.5 * (1 - min(normalized_rmse(y_true, y_pred), 1)) + 0.5 * pearson_correlation(y_true, y_pred)

# # ✅ Optuna 튜닝
# def objective(trial):
#     params = {
#         "n_estimators": trial.suggest_int("n_estimators", 300, 1000),
#         "max_depth": trial.suggest_int("max_depth", 4, 10),
#         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
#         "subsample": trial.suggest_float("subsample", 0.6, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
#         "reg_alpha": trial.suggest_float("reg_alpha", 0, 2),
#         "reg_lambda": trial.suggest_float("reg_lambda", 0, 2),
#         "random_state": 42
#     }
#     X_tr, X_val, y_tr, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)
#     model = xgb.XGBRegressor(**params)
#     model.fit(X_tr, y_tr, verbose=False)
#     y_val_pred = model.predict(X_val)
#     return -competition_score(y_val, y_val_pred)

# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=30)

# # ✅ 최종 모델 학습 및 예측
# final_model = xgb.XGBRegressor(**study.best_params)
# final_model.fit(X_train_scaled, y_train)
# submission["Inhibition"] = final_model.predict(X_test_scaled)
# submission.to_csv("submission_optuna_xgb.csv", index=False)


In [6]:


# # ✅ 라이브러리 임포트
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error
# from rdkit import Chem
# from rdkit.Chem import Descriptors, Lipinski, MACCSkeys, AllChem
# import xgboost as xgb
# import optuna
# import warnings
# warnings.filterwarnings("ignore")

# # ✅ 데이터 로딩
# path = '/content/project_data/'
# train = pd.read_csv(path + 'train.csv')
# test = pd.read_csv(path + 'test.csv')
# submission = pd.read_csv(path + 'sample_submission.csv')

# # ✅ 분자 특성 추출 함수
# def get_molecule_descriptors(smiles):
#     try:
#         mol = Chem.MolFromSmiles(smiles)
#         if mol is None: return [0] * 2232
#         basic = [
#             Descriptors.MolWt(mol), Descriptors.MolLogP(mol),
#             Descriptors.NumHAcceptors(mol), Descriptors.NumHDonors(mol),
#             Descriptors.TPSA(mol), Descriptors.NumRotatableBonds(mol),
#             Descriptors.NumAromaticRings(mol), Descriptors.NumHeteroatoms(mol),
#             Descriptors.FractionCSP3(mol), Descriptors.NumAliphaticRings(mol),
#             Lipinski.NumAromaticHeterocycles(mol), Lipinski.NumSaturatedHeterocycles(mol),
#             Lipinski.NumAliphaticHeterocycles(mol), Descriptors.HeavyAtomCount(mol),
#             Descriptors.RingCount(mol), Descriptors.NOCount(mol),
#             Descriptors.NHOHCount(mol), Descriptors.NumRadicalElectrons(mol)
#         ]
#         morgan = [int(b) for b in AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048).ToBitString()]
#         maccs = [int(b) for b in MACCSkeys.GenMACCSKeys(mol).ToBitString()]
#         return basic + morgan + maccs
#     except:
#         return [0] * 2232

# # ✅ 피처 생성
# train['features'] = train['Canonical_Smiles'].apply(get_molecule_descriptors)
# test['features'] = test['Canonical_Smiles'].apply(get_molecule_descriptors)
# X = np.array(train['features'].tolist())
# y = train['Inhibition'].values
# X_test = np.array(test['features'].tolist())

# # ✅ 정규화
# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# X_test = scaler.transform(X_test)

# # ✅ 평가 함수
# def normalized_rmse(y_true, y_pred):
#     return np.sqrt(mean_squared_error(y_true, y_pred)) / (np.max(y_true) - np.min(y_true))
# def pearson_correlation(y_true, y_pred):
#     return np.clip(np.corrcoef(y_true, y_pred)[0, 1], 0, 1)
# def competition_score(y_true, y_pred):
#     return 0.5 * (1 - min(normalized_rmse(y_true, y_pred), 1)) + 0.5 * pearson_correlation(y_true, y_pred)

# # ✅ Optuna 튜닝 (1회만, 전체 데이터 기반)
# def objective(trial):
#     params = {
#         "n_estimators": trial.suggest_int("n_estimators", 300, 1000),
#         "max_depth": trial.suggest_int("max_depth", 4, 10),
#         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
#         "subsample": trial.suggest_float("subsample", 0.6, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
#         "reg_alpha": trial.suggest_float("reg_alpha", 0, 2),
#         "reg_lambda": trial.suggest_float("reg_lambda", 0, 2),
#         "random_state": 42
#     }
#     X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
#     model = xgb.XGBRegressor(**params)
#     model.fit(X_tr, y_tr, verbose=False)
#     y_val_pred = model.predict(X_val)
#     return -competition_score(y_val, y_val_pred)

# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=30)
# best_params = study.best_params

# # ✅ KFold 앙상블 학습 + 예측
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# test_preds = np.zeros(X_test.shape[0])
# scores = []

# for fold, (tr_idx, val_idx) in enumerate(kf.split(X)):
#     print(f"📂 Fold {fold + 1}")
#     X_tr, X_val = X[tr_idx], X[val_idx]
#     y_tr, y_val = y[tr_idx], y[val_idx]

#     model = xgb.XGBRegressor(**best_params)
#     model.fit(X_tr, y_tr, verbose=False)
#     y_val_pred = model.predict(X_val)
#     score = competition_score(y_val, y_val_pred)
#     print(f"Fold {fold + 1} Score: {score:.4f}")
#     scores.append(score)

#     test_preds += model.predict(X_test) / 5  # 앙상블 평균

# # ✅ 후처리: 0~100 클리핑 + float32로 저장
# test_preds = np.clip(test_preds, 0, 100)
# submission['Inhibition'] = test_preds.astype(np.float32)
# submission.to_csv('submission_kfold_optuna.csv', index=False)

# print(f"\n✅ 평균 Score: {np.mean(scores):.4f}")
# print("📁 제출파일 저장 완료: submission_kfold_optuna.csv")


In [7]:
# import pandas as pd
# import numpy as np
# import os
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error
# from sklearn.linear_model import Ridge
# from sklearn.ensemble import RandomForestRegressor
# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
# from rdkit import Chem
# from rdkit.Chem import Descriptors, Lipinski, MACCSkeys, AllChem
# import warnings
# warnings.filterwarnings("ignore")


# # ✅ 데이터 로딩
# path = '/content/project_data/'
# train = pd.read_csv(path + 'train.csv')
# test = pd.read_csv(path + 'test.csv')
# submission = pd.read_csv(path + 'sample_submission.csv')


# # ✅ Feature 추출 함수
# def get_molecule_descriptors(smiles):
#     try:
#         mol = Chem.MolFromSmiles(smiles)
#         if mol is None:
#             return [0] * 2232

#         basic_descriptors = [
#             Descriptors.MolWt(mol),
#             Descriptors.MolLogP(mol),
#             Descriptors.NumHAcceptors(mol),
#             Descriptors.NumHDonors(mol),
#             Descriptors.TPSA(mol),
#             Descriptors.NumRotatableBonds(mol),
#             Descriptors.NumAromaticRings(mol),
#             Descriptors.NumHeteroatoms(mol),
#             Descriptors.FractionCSP3(mol),
#             Descriptors.NumAliphaticRings(mol),
#             Lipinski.NumAromaticHeterocycles(mol),
#             Lipinski.NumSaturatedHeterocycles(mol),
#             Lipinski.NumAliphaticHeterocycles(mol),
#             Descriptors.HeavyAtomCount(mol),
#             Descriptors.RingCount(mol),
#             Descriptors.NOCount(mol),
#             Descriptors.NHOHCount(mol),
#             Descriptors.NumRadicalElectrons(mol),
#         ]

#         morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
#         morgan_features = [int(bit) for bit in morgan_fp.ToBitString()]
#         maccs_fp = MACCSkeys.GenMACCSKeys(mol)
#         maccs_features = [int(bit) for bit in maccs_fp.ToBitString()]

#         return basic_descriptors + morgan_features + maccs_features
#     except:
#         return [0] * 2232

# # ✅ Feature 생성
# train['features'] = train['Canonical_Smiles'].apply(get_molecule_descriptors)
# test['features'] = test['Canonical_Smiles'].apply(get_molecule_descriptors)
# X = np.array(train['features'].tolist())
# y = train['Inhibition'].values
# X_test = np.array(test['features'].tolist())

# # ✅ 스케일링
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
# X_test_scaled = scaler.transform(X_test)

# # ✅ Metric
# def normalized_rmse(y_true, y_pred):
#     return np.sqrt(mean_squared_error(y_true, y_pred)) / (np.max(y_true) - np.min(y_true))

# def pearson_correlation(y_true, y_pred):
#     return np.clip(np.corrcoef(y_true, y_pred)[0,1], 0, 1)

# def competition_score(y_true, y_pred):
#     return 0.5 * (1 - normalized_rmse(y_true, y_pred)) + 0.5 * pearson_correlation(y_true, y_pred)

# # ✅ 모델 정의
# model = LGBMRegressor(
#     n_estimators=500, learning_rate=0.05,
#     max_depth=6, num_leaves=31,
#     subsample=0.8, colsample_bytree=0.8,
#     random_state=42
# )

# # ✅ 5-Fold 교차검증 + 예측
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# val_scores = []
# test_preds = []

# for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
#     X_train_fold, X_val_fold = X_scaled[train_idx], X_scaled[val_idx]
#     y_train_fold, y_val_fold = y[train_idx], y[val_idx]

#     model.fit(X_train_fold, y_train_fold)
#     val_pred = model.predict(X_val_fold)
#     score = competition_score(y_val_fold, val_pred)
#     val_scores.append(score)
#     print(f"[Fold {fold+1}] 검증 점수: {score:.4f}")

#     test_pred = model.predict(X_test_scaled)
#     test_preds.append(test_pred)

# # ✅ 앙상블 (평균)
# final_test_pred = np.mean(test_preds, axis=0)

# # ✅ 후처리: 0~100 클리핑
# final_test_pred = np.clip(final_test_pred, 0, 100)

# # ✅ 제출 파일 저장
# submission['Inhibition'] = final_test_pred
# submission.to_csv("submission_kfold_lgbm.csv", index=False)
# print("✅ 제출 파일 저장 완료: submission_kfold_lgbm.csv")
# print(f"평균 검증 점수: {np.mean(val_scores):.4f}")


In [ ]:
!pip install numpy==1.23.5
import os
os.kill(os.getpid(), 9)  # 런타임 재시작 (필수)


In [1]:
!pip install rdkit-pypi catboost


In [2]:


# ✅ 라이브러리 로딩
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from rdkit import Chem
from rdkit.Chem import AllChem, MACCSkeys, Descriptors, rdMolDescriptors

# ✅ 경로 설정 및 데이터 로딩
path = '/content/project_data/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

# ✅ 1. SMILES 유효성 검사
def valid_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol is not None and mol.GetNumAtoms() > 1

train = train[train['Canonical_Smiles'].apply(valid_smiles)].reset_index(drop=True)

# ✅ 2. 중복 SMILES 평균처리
train = train.groupby("Canonical_Smiles").agg({
    "Inhibition": "mean"
}).reset_index()

# ✅ 3. Canonical SMILES 재정의
def canonicalize(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return Chem.MolToSmiles(mol, canonical=True) if mol else smiles

train["Canonical_Smiles"] = train["Canonical_Smiles"].apply(canonicalize)
test["Canonical_Smiles"] = test["Canonical_Smiles"].apply(canonicalize)

# ✅ 4. 확장된 Descriptors 함수 정의
def get_expanded_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [0]*5
    return [
        Descriptors.MolMR(mol),
        rdMolDescriptors.CalcExactMolWt(mol),  # MolVol → 분자량으로 대체
        Descriptors.BalabanJ(mol),
        Descriptors.Chi0(mol),
        Descriptors.Kappa1(mol),
    ]

# ✅ 5. Feature 생성 함수 정의
def get_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(512)

    morgan = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=256)
    maccs = MACCSkeys.GenMACCSKeys(mol)
    extra = get_expanded_descriptors(smiles)

    return np.concatenate([
        np.array(morgan),
        np.array(maccs),
        np.array(extra)
    ])

# ✅ 전체 feature 변환
X = np.array([get_features(s) for s in train['Canonical_Smiles']])
X_test = np.array([get_features(s) for s in test['Canonical_Smiles']])
y = train['Inhibition'].values

# ✅ 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# ✅ 모델 정의
base_models = [
    ('xgb', XGBRegressor(n_estimators=300, learning_rate=0.1, random_state=42)),
    ('lgb', LGBMRegressor(n_estimators=300, learning_rate=0.1, random_state=42)),
    ('ridge', Ridge(alpha=1.0))
]
meta_model = Ridge()

stack_model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1
)

# ✅ KFold 학습 + 추론
kf = KFold(n_splits=5, shuffle=True, random_state=42)
preds = np.zeros(len(X_test_scaled))
for fold, (tr_idx, val_idx) in enumerate(kf.split(X_scaled, y)):
    X_train, X_val = X_scaled[tr_idx], X_scaled[val_idx]
    y_train, y_val = y[tr_idx], y[val_idx]

    stack_model.fit(X_train, y_train)
    val_pred = stack_model.predict(X_val)
    print(f'Fold {fold+1} MAE:', mean_absolute_error(y_val, val_pred))

    preds += stack_model.predict(X_test_scaled) / kf.n_splits

# ✅ 결과 저장
submission['Inhibition'] = preds
submission.to_csv('/content/submission_stack.csv', index=False)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Fold 1 MAE: 20.50232411022973


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Fold 2 MAE: 20.056287300003262


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Fold 3 MAE: 19.282389806215498


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Fold 4 MAE: 21.050464016475864
Fold 5 MAE: 21.122982024070637


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [4]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, MACCSkeys, AllChem
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

# ✅ 데이터 경로
path = '/content/project_data/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

# ✅ 유효한 SMILES 필터링
def valid_smiles(smi):
    mol = Chem.MolFromSmiles(smi)
    return mol is not None and mol.GetNumAtoms() > 1

train = train[train['Canonical_Smiles'].apply(valid_smiles)].reset_index(drop=True)

# ✅ Canonicalization
def canonicalize(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return Chem.MolToSmiles(mol, canonical=True) if mol else smiles

train['Canonical_Smiles'] = train['Canonical_Smiles'].apply(canonicalize)
test['Canonical_Smiles'] = test['Canonical_Smiles'].apply(canonicalize)

# ✅ Feature 생성
def get_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(512 + 167 + 5)

    morgan = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=512)
    morgan_np = np.array(morgan)

    maccs = MACCSkeys.GenMACCSKeys(mol)
    maccs_np = np.array(maccs)[1:]

    others = np.array([
        Descriptors.MolMR(mol),
        Descriptors.BalabanJ(mol),
        Descriptors.Chi0(mol),
        Descriptors.Chi1(mol),
        Descriptors.Kappa1(mol)
    ])

    return np.concatenate([morgan_np, maccs_np, others])

X = np.array([get_features(smi) for smi in train['Canonical_Smiles']])
X_test = np.array([get_features(smi) for smi in test['Canonical_Smiles']])
y = train['Inhibition'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

# ✅ KFold 설정
kf = KFold(n_splits=5, shuffle=True, random_state=42)

def run_stacking(meta_model, file_name):
    base_models = [
        ('xgb', XGBRegressor(n_estimators=300, random_state=42)),
        ('lgbm', LGBMRegressor(n_estimators=300, random_state=42)),
        ('cat', CatBoostRegressor(n_estimators=300, verbose=0, random_state=42))
    ]

    stack = StackingRegressor(
        estimators=base_models,
        final_estimator=meta_model,
        passthrough=True,
        n_jobs=-1
    )

    preds = np.zeros(len(X_test))
    for train_idx, val_idx in kf.split(X):
        X_tr, X_val = X[train_idx], X[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]
        stack.fit(X_tr, y_tr)
        preds += stack.predict(X_test) / kf.n_splits

    submission['Inhibition'] = preds
    submission.to_csv(file_name, index=False)

# ✅ 전략 2: Meta → LinearRegression
run_stacking(LinearRegression(), 'submission_stack_linear.csv')

# ✅ 전략 3: Meta → LGBM (CatBoost 포함)
run_stacking(LGBMRegressor(n_estimators=100, random_state=42), 'submission_stack_lgbm.csv')


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016211 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3975
[LightGBM] [Info] Number of data points in the train set: 1344, number of used features: 653
[LightGBM] [Info] Start training from score 33.391242


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015918 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3969
[LightGBM] [Info] Number of data points in the train set: 1345, number of used features: 651
[LightGBM] [Info] Start training from score 33.637152


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016477 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3972
[LightGBM] [Info] Number of data points in the train set: 1345, number of used features: 652
[LightGBM] [Info] Start training from score 33.229167


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.038461 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3978
[LightGBM] [Info] Number of data points in the train set: 1345, number of used features: 654
[LightGBM] [Info] Start training from score 32.797655


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [5]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, MACCSkeys, AllChem
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import optuna
import warnings
warnings.filterwarnings("ignore")

# ✅ 데이터 경로
path = '/content/project_data/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

# ✅ RDKit 기반 피처 생성 함수
def get_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [0]*5 + [0]*2048 + [0]*167
    desc = [
        Descriptors.MolMR(mol),
        Descriptors.BalabanJ(mol),
        Descriptors.Chi0(mol),
        Descriptors.Chi1(mol),
        Descriptors.Kappa1(mol)
    ]
    morgan = list(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048))
    maccs = list(MACCSkeys.GenMACCSKeys(mol))
    return desc + morgan + maccs

# ✅ 피처 생성
X = np.array([get_features(s) for s in train['Canonical_Smiles']])
X_test = np.array([get_features(s) for s in test['Canonical_Smiles']])
y = train['Inhibition'].values
groups = train['Canonical_Smiles'].values

# ✅ 스케일링 + PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

pca = PCA(n_components=300)
X_pca = pca.fit_transform(X_scaled)
X_test_pca = pca.transform(X_test_scaled)

# ✅ Optuna 튜닝 함수 (LGBM 예시)
def tune_lgbm(X, y, groups):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int("n_estimators", 100, 300),
            'max_depth': trial.suggest_int("max_depth", 3, 10),
            'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.3),
            'reg_lambda': trial.suggest_float("reg_lambda", 1e-4, 10.0),
            'reg_alpha': trial.suggest_float("reg_alpha", 1e-4, 10.0),
        }
        model = LGBMRegressor(**params)
        gkf = GroupKFold(n_splits=5)
        scores = []
        for train_idx, val_idx in gkf.split(X, y, groups):
            model.fit(X[train_idx], y[train_idx])
            pred = model.predict(X[val_idx])
            scores.append(mean_absolute_error(y[val_idx], pred))
        return np.mean(scores)

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)
    return study.best_params

# ✅ Optuna 튜닝 실행
best_lgbm_params = tune_lgbm(X_pca, y, groups)

# ✅ 모델 정의 및 스태킹
base_models = [
    ('lgb', LGBMRegressor(**best_lgbm_params)),
    ('xgb', XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=5)),
    ('cat', CatBoostRegressor(iterations=200, learning_rate=0.05, depth=5, verbose=0))
]

stack_model = StackingRegressor(
    estimators=base_models,
    final_estimator=Ridge(),
    cv=GroupKFold(n_splits=5)
)

# ✅ 모델 학습 및 예측
stack_model.fit(X_pca, y)
preds = stack_model.predict(X_test_pca)

submission['Inhibition'] = preds
submission.to_csv('/content/submission_optuna_stacking.csv', index=False)


[I 2025-06-25 11:35:07,281] A new study created in memory with name: no-name-89b2101f-6f4a-4713-bb30-cfdd15292de6


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005506 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76500
[LightGBM] [Info] Number of data points in the train set: 1344, number of used features: 300
[LightGBM] [Info] Start training from score 33.242107
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[I 2025-06-25 11:35:35,810] Trial 0 finished with value: 21.067014911180003 and parameters: {'n_estimators': 116, 'max_depth': 9, 'learning_rate': 0.12677114470148426, 'reg_lambda': 2.673725941372853, 'reg_alpha': 1.0128514135849045}. Best is trial 0 with value: 21.067014911180003.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005487 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76500
[LightGBM] [Info] Number of data points in the train set: 1344, number of used features: 300
[LightGBM] [Info] Start training from score 33.242107
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[I 2025-06-25 11:36:04,153] Trial 1 finished with value: 21.010951928928673 and parameters: {'n_estimators': 154, 'max_depth': 6, 'learning_rate': 0.22123397151744026, 'reg_lambda': 4.207250255989071, 'reg_alpha': 0.5292525208613081}. Best is trial 1 with value: 21.010951928928673.


...

[I 2025-06-25 11:43:04,689] Trial 18 finished with value: 20.803550509268412 and parameters: {'n_estimators': 263, 'max_depth': 4, 'learning_rate': 0.10021795838995592, 'reg_lambda': 8.04965587146662, 'reg_alpha': 6.727312087211607}. Best is trial 15 with value: 20.655339639987748.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005521 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76500
[LightGBM] [Info] Number of data points in the train set: 1344, number of used features: 300
[LightGBM] [Info] Start training from score 33.242107
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[I 2025-06-25 11:44:05,165] Trial 19 finished with value: 20.92703357190594 and parameters: {'n_estimators': 221, 'max_depth': 7, 'learning_rate': 0.1343992582817429, 'reg_lambda': 6.048407341572578, 'reg_alpha': 3.2390235186884486}. Best is trial 15 with value: 20.655339639987748.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013703 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76500
[LightGBM] [Info] Number of data points in the train set: 1681, number of used features: 300
[LightGBM] [Info] Start training from score 33.221831
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

ValueError: The 'groups' parameter should not be None.

In [7]:
print(train.columns)
print(test.columns)

Index(['ID', 'Canonical_Smiles', 'Inhibition'], dtype='object')
Index(['ID', 'Canonical_Smiles'], dtype='object')


In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, MACCSkeys, AllChem
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import optuna
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import time

# 경고 메시지 무시
warnings.filterwarnings("ignore")

print("--- Boost up AI 2025: 신약개발 경진대회 예측 파이프라인 시작 ---")
print(f"현재 시간: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())}")

# --- ✅ 0. 데이터 경로 설정 및 로드 ---
path = '/content/project_data/'
try:
    train = pd.read_csv(path + 'train.csv')
    test = pd.read_csv(path + 'test.csv')
    submission = pd.read_csv(path + 'sample_submission.csv')
    print("데이터 로드 성공.")
except FileNotFoundError:
    print(f"에러: '{path}' 경로에 데이터 파일이 없습니다. Google Colab 좌측 파일 아이콘을 통해 데이터를 업로드하거나 경로를 확인해주세요.")
    # 예시 데이터 생성 (실제 대회 데이터로 대체 필요)
    print("예시 데이터 생성 중...")
    train = pd.DataFrame({
        'ID': range(1, 1682),
        'Canonical_Smiles': ['CCO', 'CCC', 'C(C(=O)O)N', 'C1CCCCC1', 'CC(=O)Oc1ccccc1C(=O)O'] * 336 + ['CCO'],
        'Inhibition': np.random.rand(1681) * 100
    })
    test = pd.DataFrame({
        'ID': range(1, 101),
        'Canonical_Smiles': ['CN', 'CO', 'CC(C)C', 'c1ccccc1', 'NCCCN'] * 20
    })
    submission = pd.DataFrame({'ID': test['ID'], 'Inhibition': 0.0})
    print("예시 데이터 생성이 완료되었습니다. 실제 대회 데이터를 사용해주세요.")

# --- ✅ 1. 'Canonical_Smiles' 컬럼 사용 확인 ---
print("\n--- 1. 'Canonical_Smiles' 컬럼 사용 확인 ---")
train_smiles_col = 'Canonical_Smiles'
test_smiles_col = 'Canonical_Smiles'

invalid_train_smiles = train[train_smiles_col].isnull().sum()
invalid_test_smiles = test[test_smiles_col].isnull().sum()

if invalid_train_smiles > 0:
    print(f"경고: Train 데이터의 '{train_smiles_col}'에 유효하지 않은 SMILES(None)가 {invalid_train_smiles}개 있습니다. 해당 행은 피처 생성 시 0으로 처리됩니다.")
if invalid_test_smiles > 0:
    print(f"경고: Test 데이터의 '{test_smiles_col}'에 유효하지 않은 SMILES(None)가 {invalid_test_smiles}개 있습니다. 해당 행은 피처 생성 시 0으로 처리됩니다.")
else:
    print(f"'{train_smiles_col}' 및 '{test_smiles_col}' 컬럼에 유효하지 않은 (None) SMILES는 없습니다.")


# --- ✅ 2. RDKit 기반 피처 생성 ---
# RDKit의 다양한 분자 기술자(Descriptors)와 분자 지문(Fingerprints)을 추출합니다.
all_descriptor_functions = [desc_func for desc_name, desc_func in Descriptors._descList]
num_descriptors = len(all_descriptor_functions)
morgan_nbits = 2048
maccs_len = 167

def get_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [0] * num_descriptors + [0] * morgan_nbits + [0] * maccs_len

    desc_values = []
    for desc_func in all_descriptor_functions:
        try:
            # Descriptor 함수 호출 결과를 리스트에 추가
            val = desc_func(mol)
            # NaN이나 무한대 값은 0으로 처리 (PCA 오류 방지)
            if np.isnan(val) or np.isinf(val):
                desc_values.append(0)
            else:
                desc_values.append(val)
        except Exception: # 계산 불가능한 경우 (e.g., 특정 구조에만 해당) 0으로 처리
            desc_values.append(0)

    morgan = list(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=morgan_nbits))
    maccs = list(MACCSkeys.GenMACCSKeys(mol))

    return desc_values + morgan + maccs

print("\n--- 2. 피처 생성 중 (약 1~2분 소요 예상) ---")
X = np.array([get_features(s) for s in train[train_smiles_col]])
X_test = np.array([get_features(s) for s in test[test_smiles_col]])
y = train['Inhibition'].values
groups = train[train_smiles_col].values

print(f"생성된 Train 피처 차원: {X.shape}")
print(f"생성된 Test 피처 차원: {X_test.shape}")

# --- ✅ 3. 스케일링 및 차원 축소 (PCA) ---
print("\n--- 3. 스케일링 및 PCA 차원 축소 ---")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# PCA 전에 NaN 및 무한대 값 처리 (가장 중요한 수정 부분!)
# np.nan_to_num을 사용하여 NaN과 무한대 값을 0으로 바꿉니다.
X_scaled = np.nan_to_num(X_scaled, nan=0.0, posinf=0.0, neginf=0.0)
X_test_scaled = np.nan_to_num(X_test_scaled, nan=0.0, posinf=0.0, neginf=0.0)

pca = PCA(n_components=300, random_state=42)
X_pca = pca.fit_transform(X_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(f"PCA 후 Train 피처 차원: {X_pca.shape}")
print(f"PCA 후 Test 피처 차원: {X_test_pca.shape}")
print(f"PCA 누적 설명 분산 (300개 컴포넌트): {np.sum(pca.explained_variance_ratio_):.4f}")

# --- ✅ 4. StratifiedGroupKFold를 위한 타겟 값 Binning ---
print("\n--- 4. StratifiedGroupKFold를 위한 Inhibition 값 Binning ---")
bins = pd.qcut(y, q=5, labels=False, duplicates='drop')
print(f"Binning 결과 (상위 5개): {bins[:5]}")
print(f"각 Bin별 데이터 개수:\n{pd.Series(bins).value_counts().sort_index()}")


# --- ✅ 5. Optuna를 이용한 Base 모델 하이퍼파라미터 튜닝 ---
def tune_model(X_data, y_data, groups_data, bins_data, model_type, n_trials=50, cv_folds=5):
    def objective(trial):
        if model_type == 'lgbm':
            params = {
                'objective': 'mae',
                'metric': 'mae',
                'n_estimators': trial.suggest_int("n_estimators", 100, 1000),
                'max_depth': trial.suggest_int("max_depth", 3, 15),
                'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.2),
                'reg_lambda': trial.suggest_float("reg_lambda", 1e-3, 10.0),
                'reg_alpha': trial.suggest_float("reg_alpha", 1e-3, 10.0),
                'colsample_bytree': trial.suggest_float("colsample_bytree", 0.6, 1.0),
                'subsample': trial.suggest_float("subsample", 0.6, 1.0),
                'n_jobs': -1,
                'random_state': 42,
                'verbose': -1,
            }
            model = LGBMRegressor(**params)
        elif model_type == 'xgb':
            params = {
                'objective': 'reg:absoluteerror',
                'eval_metric': 'mae',
                'n_estimators': trial.suggest_int("n_estimators", 100, 1000),
                'max_depth': trial.suggest_int("max_depth", 3, 15),
                'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.2),
                'subsample': trial.suggest_float("subsample", 0.6, 1.0),
                'colsample_bytree': trial.suggest_float("colsample_bytree", 0.6, 1.0),
                'gamma': trial.suggest_float("gamma", 1e-3, 10.0),
                'reg_lambda': trial.suggest_float("lambda", 1e-3, 10.0),
                'reg_alpha': trial.suggest_float("alpha", 1e-3, 10.0),
                'n_jobs': -1,
                'random_state': 42,
                'tree_method': 'hist',
            }
            model = XGBRegressor(**params)
        elif model_type == 'cat':
            params = {
                'loss_function': 'MAE',
                'iterations': trial.suggest_int("iterations", 100, 1000),
                'depth': trial.suggest_int("depth", 3, 10),
                'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.2),
                'l2_leaf_reg': trial.suggest_float("l2_leaf_reg", 1e-3, 10.0),
                'border_count': trial.suggest_int("border_count", 32, 255),
                'verbose': 0,
                'random_seed': 42,
            }
            model = CatBoostRegressor(**params)
        else:
            raise ValueError(f"알 수 없는 모델 타입: {model_type}")

        sgkf = StratifiedGroupKFold(n_splits=cv_folds, shuffle=True, random_state=42)
        scores = []
        for train_idx, val_idx in sgkf.split(X_data, bins_data, groups_data):
            model.fit(X_data[train_idx], y_data[train_idx])
            pred = model.predict(X_data[val_idx])
            scores.append(mean_absolute_error(y_data[val_idx], pred))
        return np.mean(scores)

    print(f"\n--- 5. Optuna 튜닝 시작: {model_type} (n_trials={n_trials}) ---")
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    print(f"최적 MAE: {study.best_value:.4f}")
    print(f"최적 파라미터: {study.best_params}")
    return study.best_params

best_lgbm_params = tune_model(X_pca, y, groups, bins, 'lgbm')
best_xgb_params = tune_model(X_pca, y, groups, bins, 'xgb')
best_cat_params = tune_model(X_pca, y, groups, bins, 'cat')

# --- ✅ 6. Stacking Regressor 모델 정의 및 학습 ---
print("\n--- 6. Stacking Regressor 모델 정의 및 학습 ---")
base_models = [
    ('lgb', LGBMRegressor(**best_lgbm_params)),
    ('xgb', XGBRegressor(**best_xgb_params)),
    ('cat', CatBoostRegressor(**best_cat_params, verbose=0))
]

stack_model = StackingRegressor(
    estimators=base_models,
    final_estimator=Ridge(random_state=42),
    cv=StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42),
    n_jobs=-1
)

print("Stacking 모델 학습 시작...")
stack_model.fit(X_pca, y)
print("Stacking 모델 학습 완료.")

# --- ✅ 7. 예측 및 후처리 ---
print("\n--- 7. 예측 및 후처리 ---")
preds = stack_model.predict(X_test_pca)

preds[preds < 0] = 0
preds[preds > 100] = 100

print(f"최종 예측값 평균: {np.mean(preds):.2f}")
print(f"최종 예측값 표준편차: {np.std(preds):.2f}")
print(f"최종 예측값 최소: {np.min(preds):.2f}")
print(f"최종 예측값 최대: {np.max(preds):.2f}")

# --- ✅ 8. 성능 검증 및 시각화 (결과 미리 확인) ---
print("\n--- 8. 성능 검증 및 시각화 ---")

# 8.1 Stacking 모델의 교차 검증 MAE 확인
print("Stacking 모델 교차 검증 MAE 계산 중...")
cv_scores = []
sgkf_final_eval = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(sgkf_final_eval.split(X_pca, bins, groups)):
    print(f"  - 폴드 {fold+1}/{sgkf_final_eval.n_splits} 평가 중...")

    fold_stack_model = StackingRegressor(
        estimators=base_models,
        final_estimator=Ridge(random_state=42),
        cv=StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42),
        n_jobs=-1
    )
    fold_stack_model.fit(X_pca[train_idx], y[train_idx])
    val_preds = fold_stack_model.predict(X_pca[val_idx])

    val_preds[val_preds < 0] = 0
    val_preds[val_preds > 100] = 100

    mae = mean_absolute_error(y[val_idx], val_preds)
    cv_scores.append(mae)
    print(f"    MAE: {mae:.4f}")

print(f"\nStacking 모델 최종 교차 검증 MAE: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")

# 8.2 예측값 분포 및 잔차 분석 시각화
print("예측값 분포 및 잔차 분석 시각화 생성 중...")

train_preds = stack_model.predict(X_pca)
train_preds[train_preds < 0] = 0
train_preds[train_preds > 100] = 100

plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
sns.histplot(y, color='blue', label='Actual Inhibition (%) (Train)', kde=True, stat='density', linewidth=0, alpha=0.6)
sns.histplot(train_preds, color='orange', label='Predicted Inhibition (%) (Train)', kde=True, stat='density', linewidth=0, alpha=0.5)
sns.histplot(preds, color='green', label='Predicted Inhibition (%) (Test)', kde=True, stat='density', linewidth=0, alpha=0.5)
plt.title('Distribution of Inhibition (%) Values')
plt.xlabel('Inhibition (%)')
plt.ylabel('Density')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.subplot(1, 3, 2)
residuals = y - train_preds
sns.histplot(residuals, kde=True, color='purple', stat='density', linewidth=0)
plt.title('Distribution of Residuals (Actual - Predicted) on Train Data')
plt.xlabel('Residuals')
plt.ylabel('Density')
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.subplot(1, 3, 3)
plt.scatter(y, residuals, alpha=0.3, color='teal')
plt.axhline(0, color='red', linestyle='--', label='Zero Residual')
plt.title('Residuals vs. Actual Inhibition (%) on Train Data')
plt.xlabel('Actual Inhibition (%)')
plt.ylabel('Residuals (Actual - Predicted)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# --- ✅ 9. 제출 파일 생성 ---
print("\n--- 9. 제출 파일 생성 ---")
submission['Inhibition'] = preds
submission.to_csv('/content/submission_optuna_stacking_enhanced.csv', index=False)
print("submission_optuna_stacking_enhanced.csv 파일이 성공적으로 생성되었습니다.")
print(f"최종 예측 결과 파일: /content/submission_optuna_stacking_enhanced.csv")
print(f"파이프라인 종료 시간: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())}")
print("\n--- Boost up AI 2025: 신약개발 경진대회 예측 파이프라인 완료 ---")

--- Boost up AI 2025: 신약개발 경진대회 예측 파이프라인 시작 ---
현재 시간: 2025-06-25 11:57:12
데이터 로드 성공.

--- 1. 'Canonical_Smiles' 컬럼 사용 확인 ---
'Canonical_Smiles' 및 'Canonical_Smiles' 컬럼에 유효하지 않은 (None) SMILES는 없습니다.

--- 2. 피처 생성 중 (약 1~2분 소요 예상) ---
생성된 Train 피처 차원: (1681, 2423)
생성된 Test 피처 차원: (100, 2423)

--- 3. 스케일링 및 PCA 차원 축소 ---


[I 2025-06-25 11:57:50,029] A new study created in memory with name: no-name-6f8fc410-2839-4e9e-a531-dfa0817eb5b3


PCA 후 Train 피처 차원: (1681, 300)
PCA 후 Test 피처 차원: (100, 300)
PCA 누적 설명 분산 (300개 컴포넌트): 0.5438

--- 4. StratifiedGroupKFold를 위한 Inhibition 값 Binning ---
Binning 결과 (상위 5개): [1 0 0 4 1]
각 Bin별 데이터 개수:
0    337
1    336
2    338
3    334
4    336
Name: count, dtype: int64

--- 5. Optuna 튜닝 시작: lgbm (n_trials=50) ---


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-06-25 11:58:54,139] Trial 0 finished with value: 20.693897866256012 and parameters: {'n_estimators': 437, 'max_depth': 15, 'learning_rate': 0.14907884894416698, 'reg_lambda': 5.986986183486169, 'reg_alpha': 1.5610303857839227, 'colsample_bytree': 0.662397808134481, 'subsample': 0.6232334448672797}. Best is trial 0 with value: 20.693897866256012.
[I 2025-06-25 12:00:44,594] Trial 1 finished with value: 20.575353355434252 and parameters: {'n_estimators': 880, 'max_depth': 10, 'learning_rate': 0.14453378978124864, 'reg_lambda': 0.20682435846372868, 'reg_alpha': 9.699128611767781, 'colsample_bytree': 0.9329770563201687, 'subsample': 0.6849356442713105}. Best is trial 1 with value: 20.575353355434252.
[I 2025-06-25 12:01:06,801] Trial 2 finished with value: 20.495512391208077 and parameters: {'n_estimators': 263, 'max_depth': 5, 'learning_rate': 0.06780602616231217, 'reg_lambda': 5.2480395598907466, 'reg_alpha': 4.320018241402516, 'colsample_bytree': 0.7164916560792167, 'subsample':

[I 2025-06-25 12:51:04,009] A new study created in memory with name: no-name-ac5edfdb-b600-47fb-b99b-f3adf78ba430


[I 2025-06-25 12:51:04,002] Trial 49 finished with value: 20.320767196805008 and parameters: {'n_estimators': 781, 'max_depth': 8, 'learning_rate': 0.042127090271012683, 'reg_lambda': 0.015498667915740505, 'reg_alpha': 2.596964974125532, 'colsample_bytree': 0.6198049919810044, 'subsample': 0.8079183100215245}. Best is trial 13 with value: 20.06297442657618.
최적 MAE: 20.0630
최적 파라미터: {'n_estimators': 783, 'max_depth': 8, 'learning_rate': 0.0392550683118172, 'reg_lambda': 7.719468199788334, 'reg_alpha': 5.329317320726376, 'colsample_bytree': 0.608105951159242, 'subsample': 0.7371197153065265}

--- 5. Optuna 튜닝 시작: xgb (n_trials=50) ---


  0%|          | 0/50 [00:00<?, ?it/s]